# TensorFlow 시작하기
[원본 문서](https://www.tensorflow.org/get_started/get_started)

이 안내서는 TensorFlow에서 프로그래밍을 시작하도록 안내합니다. 이 안내서를 사용하기 전에, [TensorFlow를 설치하십시오](https://www.tensorflow.org/install/). 이 가이드를 최대한 활용하려면, 다음 사항들을 알아야 합니다:

* 파이썬(Python)으로 프로그래밍하는 법.
* 적어도 배열에 대한 약간의 지식.
* 이상적으로는, 기계 학습에 관한 것입니다. 그러나, 기계 학습에 대해 거의 또는 전혀 알지 못하는 경우에도, 이것이 그래도 읽어야 할 첫 번째 가이드입니다.

TensorFlow는 여러 API들을 제공합니다. 최저 수준의 API--TensorFlow Core--는 완벽한 프로그래밍 제어를 제공합니다. TensorFlow Core는 기계 학습 연구자 및 모델을 정밀하게 제어해야 하는 사람들을 위해 권장됩니다. 높은 수준의 API는 TensorFlow Core 위에 구축됩니다. 이러한 상위 수준의 API는 일반적으로 TensorFlow Core보다 배우고 사용하기가 쉽습니다. 또한 상위 수준의 API는 반복적인 작업을 여러 사용자간에 보다 쉽고 일관되게 만듭니다. tf.estimator와 같은 고급 API를 통해 데이터 셋, 추정자(estimator), 학습 및 추론을 처리할 수 있습니다.

이 안내서는 TensorFlow Core에 대한 자습서로 시작됩니다. 나중에, tf.estimator에서 동일한 모델을 구현하는 방법을 보여줍니다. TensorFlow Core의 원리를 아는 것은 보다 간결한 고수준의 API를 사용할 때 내부적으로 어떤 일이 일어나는 지에 대한 훌륭한 개념 모델을 제공합니다.

# 텐서(Tensors)

TensorFlow 데이터의 중심 단위는 **텐서(tensor)**입니다. 텐서는 임의의 차원 배열 구조(shape)인 원시 값들(primitive values)의 집합으로 구성됩니다. 텐서의 **순위(rank)**는 차원 수입니다. 다음은 텐서들의 몇 가지 예입니다:

In [2]:
3 # a rank 0 tensor; this is a scalar with shape []

3

In [3]:
[1., 2., 3.] # a rank 1 tensor; this is a vector with shape [3]

[1.0, 2.0, 3.0]

In [4]:
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]

[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]

In [5]:
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]

[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]

## TensorFlow 핵심 자습서
<hr/>

### TensorFlow 가져 오기

TensorFlow 프로그램에 대한 표준 import 문은 다음과 같습니다:

In [6]:
import tensorflow as tf

이렇게하면 파이썬은 TensorFlow의 모든 클래스, 메소드 및 심볼에 접근 할 수 있습니다. 대부분의 문서에서는 이미 이 작업을 수행했다고 가정합니다.

### 연산 그래프

TensorFlow 핵심 프로그램은 두 개의 분리된 부분으로 구성되어 있다고 생각할 수 있습니다:

1. 연산 그래프의 작성.
2. 연산 그래프의 실행.

**연산 그래프**는 일련의 TensorFlow 작업을 노드 그래프로 배열한 것입니다. 간단한 연산 그래프를 작성해 봅시다. 각 노드는 0 이상의 텐서를 입력으로 사용하고 텐서를 출력으로 생성합니다. 노드의 한 유형은 상수입니다. 모든 TensorFlow 상수와 마찬가지로, 입력을 받지 않으며, 내부적으로 저장하는 값을 출력합니다. 다음과 같이 두 개의 부동 소수점 텐서 `node1`과 `node2`를 생성 할 수 있습니다:

In [7]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


마지막 출력 문의 결과는 ↑

노드의 출력이 아마도 예상했을 값 `3.0`과 `4.0`로 출력되지 않는다는 점에 주목하십시오. 대신, 평가시에, 각각 3.0과 4.0를 생성하는 노드입니다. 노드를 실제로 평가하려면 **세션(session)** 내에서 연산 그래프를 실행해야합니다. 세션은 TensorFlow 런타임의 제어(control)와 상태(state)를 캡슐화합니다.

다음 코드는 `Session` 객체를 만든 다음 `run` 메소드를 호출하여 `node1`과 `node2`를 계산할 수있는 연산 그래프를 실행합니다. 다음과 같이 세션에서 연산 그래프를 실행하면:

In [8]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


예상했던 값인 3.0과 4.0가 나타납니다: ↑

`Tensor` 노드들을 작업(작업 또한 노드입니다)들과 결합하여 보다 복잡한 연산을 만들 수 있습니다. 예를 들어, 두 개의 상수 노드를 추가하여 다음과 같이 새 그래프를 생성 할 수 있습니다:

In [9]:
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

node3: Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3): 7.0


마지막 두 개의 출력 문의 결과입니다 ↑

TensorFlow는 연산 그래프의 그림을 표시하는 TensorBoard라는 유틸리티를 제공합니다. 다음은 TensorBoard가 그래프를 시각화하는 방법을 보여주는 스크린 샷입니다:
<img alt="TensorBoard screenshot" src="https://www.tensorflow.org/images/getting_started_add.png">
이대로, 이 그래프는 항상 동일한 결과를 산출하기 때문에 특별히 흥미롭지는 않습니다. **자리 표시자(placeholder)**라고 하는 외부 입력을 허용하도록 그래프를 파라미터화(parameterized) 할 수 있습니다. **자리 표시자**는 나중에 값을 제공하겠다는 약속입니다.

In [10]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

위의 세 줄은 함수 또는 람다와 비슷하지만 두 개의 입력 파라미터들(a와 b)을 정의한 다음 해당 파라미터들에 대한 연산을 정의합니다. [run 메소드](https://www.tensorflow.org/api_docs/python/tf/Session#run)에 feed_dict 인수를 사용하여 구체적인 값을 자리 표시자에 공급함으로써 이 그래프가 여러 입력에 대해 평가할 수 있도록 합니다:

In [11]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

7.5
[ 3.  7.]


출력 결과 ↑

텐서보드(TensorBoard)에서 그래프는 다음과 같습니다:
<img alt="TensorBoard screenshot" src="https://www.tensorflow.org/images/getting_started_adder.png">
다른 연산을 추가하여 연산 그래프를 더 복잡하게 만들 수 있습니다. 예를 들어,

In [12]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

22.5


은 출력을 만듭니다 ↑

앞의 연산 그래프는 TensorBoard에서 다음과 같이 나타납니다:
<img alt="TensorBoard screenshot" src="https://www.tensorflow.org/images/getting_started_triple.png">
기계 학습에서 우리는 일반적으로 위와 같은 임의의 입력을 취할 수 있는 모델을 원할 것입니다. 모델을 학습 가능하게 만들려면, 동일한 입력으로 새로운 출력을 얻기 위해 그래프를 수정할 수 있어야 합니다. **변수**를 사용하면 그래프에 학습 가능한 파라미터를 추가할 수 있습니다. 그것들은 타입과 초기 값으로 구성됩니다:

In [13]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

상수는 `tf.constant`를 호출할 때 초기화되며 값은 절대로 변경될 수 없습니다. 반대로 `tf.Variable`을 호출하면 변수가 초기화되지 않습니다. TensorFlow 프로그램의 모든 변수를 초기화하려면 다음과 같이 특수한 작업을 명시적으로 호출해야합니다:

In [14]:
init = tf.global_variables_initializer()
sess.run(init)

`init`이 TensorFlow 하위 그래프에 모든 전역 변수를 초기화하는 기회임을 인식하는 것이 중요합니다. `sess.run`을 호출할 때까지 변수는 초기화되지 않습니다.

`x`는 자리 표시자이므로, 다음과 같이 `x`의 여러 값들에 대해 `linear_model`을 동시에 평가할 수 있습니다:

In [15]:
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


출력을 만듭니다 ↑

우리는 모델을 만들었지만, 아직 얼마나 좋은지 모릅니다. 교육 데이터에 대한 모델을 평가하려면, 원하는 값을 제공하기 위해 <mark>y</mark> 자리 표시자가 필요하며, 손실 함수(loss function)를 작성해야 합니다.

손실 함수는 제공된 모델로부터 현재 모델이 얼마나 떨어져 있는지를 측정합니다. 현재 모델과 제공된 데이터 사이의 델타 제곱을 합한 선형 회귀분석(linear regression)에 표준 손실 모델(standard loss model)을 사용합니다. `linear_model - y`는 각 요소에 해당하는 예의 오차 델타 벡터를 만듭니다. `tf.square`를 호출하여 오차를 제곱합니다. 그런 다음, `tf.reduce_sum`을 사용해, 모든 오차의 제곱을 합함으로써 모든 예들의 오차를 대표(abstract)하는 단일 스칼라를 만듭니다:

In [16]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


손실 값을 만듭니다 ↑

`W`와 `b`를 정확한 값들인 -1과 1로 재할당하여 수동으로 향상시킬 수 있습니다. 변수는 `tf.Variable`을 통해 초기화되지만 `tf.assign`과 같은 연산을 사용해 변경할 수 있습니다. 예를 들어, `W=-1`과 `b=1`은 우리 모델에서 최적의 파라미터들입니다. `W`와 `b`를 적절하게 변경할 수 있습니다:

In [17]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


마지막 출력은 손실이 0임을 보여줍니다. ↑

우리는 `W`와 `b`의 "정확한" 값들을 추측했지만, 기계 학습의 요점은 올바른 모델 파라미터들을 자동으로 찾는 것입니다. 다음 절에서 이것을 이루는 방법을 보여주겠습니다.

## tf.train API
<hr/>

기계 학습에 대한 완전한 논의는 본 자습서의 범위를 벗어납니다. 하지만, TensorFlow는 손실 함수를 최소화하기 위해 각 변수를 조금씩 변경하는 **옵티마이저(optimizer)**들을 제공합니다. 가장 간단한 옵티마이저는 **경사 하강(gradient descent)**입니다. 해당 변수의 손실에 대한 미분 크기에 따라 각 변수를 수정합니다. 일반적으로, 수작업으로 계산하는 것은 지루하고 오류가 발생하기 쉽습니다. 따라서, TensorFlow는 `tf.gradients` 함수를 사용하여 주어진 모델에 대한 기술만으로 미분값을 자동으로 생성할 수 있습니다. 단순화 하자면, 옵티마이저가 일반적으로 이를 수행합니다. 예를 들어,

In [18]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [19]:
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
  sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


results in the final model parameters: ↑

Now we have done actual machine learning! Although doing this simple linear regression doesn't require much TensorFlow core code, more complicated models and methods to feed data into your model necessitate more code. Thus TensorFlow provides higher level abstractions for common patterns, structures, and functionality. We will learn how to use some of these abstractions in the next section.

### Complete program

The completed trainable linear regression model is shown here:

In [20]:
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


When run, it produces ↑

Notice that the loss is a very small number (very close to zero). If you run this program, your loss may not be the exact same because the model is initialized with pseudorandom values.

This more complicated program can still be visualized in TensorBoard TensorBoard final model visualization
<img alt="TensorBoard final model visualization" src="https://www.tensorflow.org/images/getting_started_final.png">

## tf.estimator
<hr/>

<mark>tf.estimator</mark> is a high-level TensorFlow library that simplifies the mechanics of machine learning, including the following:

* running training loops
* running evaluation loops
* managing data sets

tf.estimator defines many common models.



### Basic usage

Notice how much simpler the linear regression program becomes with <mark>tf.estimator</mark>:

In [22]:
import tensorflow as tf
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np

# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp1ovxbcgn', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp1ovxbcgn/model.ckpt.
INFO:tensorflow:loss = 13.0, step = 1
INFO:tensorflow:global_step/sec: 1253.18
INFO:tensorflow:loss = 0.404879, step = 101 (0.083 sec)
INFO:tensorflow:global_step/sec: 1367.07
INFO:tensorflow:loss = 0.0710876, step = 201 (0.074 sec)
INFO:tensorflow:global_step/sec: 1332.47
INFO:tensorflow:loss = 0.0199711, step = 301 (0.074 sec)
INFO:tensorflow:global_step/sec: 1223.98
INFO:tensorflow:loss = 0.00589484, step = 401 (0.081 sec)
INFO:tensorflow:global_step/sec: 1282.52
INFO:tensorflow:loss = 0.000747879, step = 501 (0.077 sec)
INFO:tensor

When run, it produces ↑

Notice how our eval data has a higher loss, but it is still close to zero. That means we are learning properly.

### A custom model

<mark>tf.estimator</mark> does not lock you into its predefined models. Suppose we wanted to create a custom model that is not built into TensorFlow. We can still retain the high level abstraction of data set, feeding, training, etc. of <mark>tf.estimator</mark>. For illustration, we will show how to implement our own equivalent model to <mark>LinearRegressor</mark> using our knowledge of the lower level TensorFlow API.

To define a custom model that works with <mark>tf.estimator</mark>, we need to use <mark>tf.estimator.Estimator</mark>. <mark>tf.estimator.LinearRegressor</mark> is actually a sub-class of <mark>tf.estimator.Estimator</mark>. Instead of sub-classing <mark>Estimator</mark>, we simply provide Estimator a function <mark>model_fn</mark> that tells <mark>tf.estimator</mark> how it can evaluate predictions, training steps, and loss. The code is as follows:

In [24]:
import numpy as np
import tensorflow as tf

# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W * features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # EstimatorSpec connects subgraphs we built to the
  # appropriate functionality.
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpwns0ghnb', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpwns0ghnb/model.ckpt.
INFO:tensorflow:loss = 17.3073748566, step = 1
INFO:tensorflow:global_step/sec: 1339.49
INFO:tensorflow:loss = 0.0416367058214, step = 101 (0.078 sec)
INFO:tensorflow:global_step/sec: 1318.66
INFO:tensorflow:loss = 0.00252774516377, step = 201 (0.075 sec)
INFO:tensorflow:global_step/sec: 1363.17
INFO:tensorflow:loss = 5.93164722352e-05, step = 301 (0.073 sec)
INFO:tensorflow:global_step/sec: 1441.82
INFO:tensorflow:loss = 1.58568519768e-05, step = 401 (0.070 sec)
INFO:tensorflow:global_step/sec: 1343.92
INFO:tensorflow:loss = 4.4833211

When run, it produces ↑

Notice how the contents of the custom <mark>model_fn()</mark> function are very similar to our manual model training loop from the lower level API.

## Next steps
<hr/>

Now you have a working knowledge of the basics of TensorFlow. We have several more tutorials that you can look at to learn more. If you are a beginner in machine learning see [MNIST for beginners](https://www.tensorflow.org/get_started/mnist/beginners), otherwise see [Deep MNIST for experts](https://www.tensorflow.org/get_started/mnist/pros).
<hr/>
Except as otherwise noted, the content of this page is licensed under the Creative Commons Attribution 3.0 License, and code samples are licensed under the Apache 2.0 License. For details, see our Site Policies. Java is a registered trademark of Oracle and/or its affiliates.

Last updated 8월 17, 2017.